In [1]:
from pathlib import Path

root_dir = Path("data", "adaptation")
static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")
tif_files = list(hazard_path.glob("*.tif"))

#calling to the cost file : this should be an intermediate output from previous calculations
AO_input_data = root_dir.joinpath("data")

In [ ]:
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,HazardSection,)
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkSection)

_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       #Used to specify the shapefile name of the (road) network to do the analysis with, when creating a network from a shapefile.
    primary_file = [network_path.joinpath("network.shp")], #specify in the RA2CE folder setup where the network is locates
    file_id = "rfid_c",
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
    hazard_field_name= ['waterdepth'],
    aggregate_wl = AggregateWlEnum.MAX,
    hazard_crs = 'EPSG:4326' 
)

#pass the specified sections as arguments for configuration
_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )



In [ ]:

handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()
handler.run_analysis()